# Student Performance vs Funding

#### Data sources: https://www.kaggle.com/noriuk/us-educational-finances?select=states.csv

### Import Dependencies and Setup for the analyze

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import seaborn as sns
import warnings

# Study data files paths
average_scores_path = "Input_Data/naep.csv"
fed_revenue_path = "Input_Data/states.csv"

# Read the score data and the revenue results
scores_data = pd.read_csv(average_scores_path)
revenue_data = pd.read_csv(fed_revenue_path)

In [2]:
# Combine the data into a single dataset
merged_data_df = pd.merge(scores_data, revenue_data, how = "inner", 
                          left_on=['STATE','YEAR'], right_on=['STATE', 'YEAR'])

# Convert to numeric
merged_data_df['AVG_SCORE'] = merged_data_df['AVG_SCORE'].apply(pd.to_numeric, errors = 'coerce')
merged_data_df['TOTAL_REVENUE'] = merged_data_df['TOTAL_REVENUE'].apply(pd.to_numeric, errors = 'coerce')
merged_data_df['INSTRUCTION_EXPENDITURE'] = merged_data_df['INSTRUCTION_EXPENDITURE'].apply(pd.to_numeric, errors = 'coerce' )

merged_data_df = merged_data_df.drop(columns=['FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE', 
                                              'TOTAL_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE', 
                                              'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE'])

merged_data_df = merged_data_df.dropna()

# Display the data table for preview
merged_data_df

,YEAR,STATE,AVG_SCORE,TEST_SUBJECT,TEST_YEAR,ENROLL,TOTAL_REVENUE,INSTRUCTION_EXPENDITURE
0,2015,Alabama,230.977423,Mathematics,4,734974.0,7360222,3817479
1,2015,Alabama,266.555318,Mathematics,8,734974.0,7360222,3817479
2,2015,Alabama,217.049090,Reading,4,734974.0,7360222,3817479
3,2015,Alabama,258.753761,Reading,8,734974.0,7360222,3817479
4,2015,Alaska,236.328417,Mathematics,4,130755.0,2920986,1484610
...,...,...,...,...,...,...,...,...
2035,1994,Virginia,213.457286,Reading,4,1044721.0,6125739,3271356
2036,1994,Washington,212.695386,Reading,4,915952.0,5739012,2918426
2037,1994,West Virginia,213.146935,Reading,4,313750.0,1854403,1051169
2038,1994,Wisconsin,224.174545,Reading,4,843747.0,5682439,3257526


In [3]:
# Calculate the Total revenue per Student
# Note: The total revenue is the same regardless the grade or subject

revenue_per_student = round((merged_data_df["TOTAL_REVENUE"]*1000)/merged_data_df["ENROLL"],2)
# revenue_per_student.head()

merged_data_df["TOTAL_REV_PER_STUDENT"] = revenue_per_student
merged_data_df

,YEAR,STATE,AVG_SCORE,TEST_SUBJECT,TEST_YEAR,ENROLL,TOTAL_REVENUE,INSTRUCTION_EXPENDITURE,TOTAL_REV_PER_STUDENT
0,2015,Alabama,230.977423,Mathematics,4,734974.0,7360222,3817479,10014.26
1,2015,Alabama,266.555318,Mathematics,8,734974.0,7360222,3817479,10014.26
2,2015,Alabama,217.049090,Reading,4,734974.0,7360222,3817479,10014.26
3,2015,Alabama,258.753761,Reading,8,734974.0,7360222,3817479,10014.26
4,2015,Alaska,236.328417,Mathematics,4,130755.0,2920986,1484610,22339.38
...,...,...,...,...,...,...,...,...,...
2035,1994,Virginia,213.457286,Reading,4,1044721.0,6125739,3271356,5863.52
2036,1994,Washington,212.695386,Reading,4,915952.0,5739012,2918426,6265.63
2037,1994,West Virginia,213.146935,Reading,4,313750.0,1854403,1051169,5910.45
2038,1994,Wisconsin,224.174545,Reading,4,843747.0,5682439,3257526,6734.77


In [4]:
# Calculate the Instruction spending per Student for 2015
# Note: The instruction expenditure is the same regardless the grade or subject

spending_per_student = round((merged_data_df["INSTRUCTION_EXPENDITURE"]*1000)/merged_data_df["ENROLL"],2)
# spending_per_student.head()

merged_data_df["INSTRUCTION_EXPENDITURE_PER_STUDENT"] = spending_per_student
merged_data_df

,YEAR,STATE,AVG_SCORE,TEST_SUBJECT,TEST_YEAR,ENROLL,TOTAL_REVENUE,INSTRUCTION_EXPENDITURE,TOTAL_REV_PER_STUDENT,INSTRUCTION_EXPENDITURE_PER_STUDENT
0,2015,Alabama,230.977423,Mathematics,4,734974.0,7360222,3817479,10014.26,5194.03
1,2015,Alabama,266.555318,Mathematics,8,734974.0,7360222,3817479,10014.26,5194.03
2,2015,Alabama,217.049090,Reading,4,734974.0,7360222,3817479,10014.26,5194.03
3,2015,Alabama,258.753761,Reading,8,734974.0,7360222,3817479,10014.26,5194.03
4,2015,Alaska,236.328417,Mathematics,4,130755.0,2920986,1484610,22339.38,11354.14
...,...,...,...,...,...,...,...,...,...,...
2035,1994,Virginia,213.457286,Reading,4,1044721.0,6125739,3271356,5863.52,3131.32
2036,1994,Washington,212.695386,Reading,4,915952.0,5739012,2918426,6265.63,3186.22
2037,1994,West Virginia,213.146935,Reading,4,313750.0,1854403,1051169,5910.45,3350.34
2038,1994,Wisconsin,224.174545,Reading,4,843747.0,5682439,3257526,6734.77,3860.79


In [5]:
# Group merged data by the test subject
test_subject = merged_data_df.groupby(['TEST_SUBJECT'])
# test_subject.head()

### 2015 Reveune and Spending analysis 
2015 is the most recent year

In [6]:
# 2015 dataframe
year_2015 = merged_data_df[(merged_data_df['YEAR'] == 2015)]

year_2015.head()

,YEAR,STATE,AVG_SCORE,TEST_SUBJECT,TEST_YEAR,ENROLL,TOTAL_REVENUE,INSTRUCTION_EXPENDITURE,TOTAL_REV_PER_STUDENT,INSTRUCTION_EXPENDITURE_PER_STUDENT
0,2015,Alabama,230.977423,Mathematics,4,734974.0,7360222,3817479,10014.26,5194.03
1,2015,Alabama,266.555318,Mathematics,8,734974.0,7360222,3817479,10014.26,5194.03
2,2015,Alabama,217.049090,Reading,4,734974.0,7360222,3817479,10014.26,5194.03
3,2015,Alabama,258.753761,Reading,8,734974.0,7360222,3817479,10014.26,5194.03
4,2015,Alaska,236.328417,Mathematics,4,130755.0,2920986,1484610,22339.38,11354.14


In [ ]:
# Total revenue by State in 2015, presenting in barplot
rev_sort_2015 = year_2015.sort_values("TOTAL_REVENUE", ascending = False)
rev_by_billion = round(rev_sort_2015["TOTAL_REVENUE"]/1000000,0)
rev_sort_2015["TOTAL_REVENUE"] = rev_by_billion

sns.set_theme(color_codes = True)

f, ax = plt.subplots(figsize = (10, 15))
 
sns.barplot(x = "TOTAL_REVENUE", y = "STATE", data = rev_sort_2015, ax = ax)
ax.set(xlabel = "TOTAL BUDGET\n(Billion)") 
plt.title("Total Revenue\nYear 2015", fontsize = 14, fontweight = "bold")

plt.tight_layout(h_pad = 1)

plt.savefig("Output_images/State_Total_Rev")


In [ ]:
# Total revenue vs Instruction expenditure per student

rev_per_stu_sort_2015 = year_2015.sort_values("TOTAL_REV_PER_STUDENT", ascending = False)

f, ax = plt.subplots(figsize = (10, 15))

sns.set_color_codes("pastel")
ax = sns.barplot(x = "TOTAL_REV_PER_STUDENT", y = "STATE", 
                 label = "REVENUE PER STUDENT", data = rev_per_stu_sort_2015, color = "orange", ax = ax)


sns.set_color_codes("muted")
ax = sns.barplot(x = "INSTRUCTION_EXPENDITURE_PER_STUDENT", y = "STATE", 
                 label = "INSTRUCTION EXPENDITURE PER STUDENT", data = rev_per_stu_sort_2015, color = "crimson", ax = ax)

handles, labels = ax.get_legend_handles_labels()

ax.legend(ncol = 1, loc = "center right", frameon = True, edgecolor = "w", borderaxespad = 5)

ax.set(xlim = (0, 30500), ylabel = "",
       xlabel = "DOLLAR PER STUDENT")
plt.title("Revenue & Instruction Expenditure Per Student\nYear 2015", fontsize = 14, fontweight = "bold")

sns.despine(bottom = True)
plt.tight_layout(h_pad = 1)

plt.savefig("Output_images/State_Rev_Exp_Per_Student")


### 2015 Funding vs Average Reading Scores

In [ ]:
# Total revenue vs Instruction expenditure per student
# Sorted by 8th grade Reading avg score

year_2015_8th_read = year_2015[(year_2015['TEST_YEAR'] == 8) & (year_2015['TEST_SUBJECT'] == "Reading")]
sort_8th_read = year_2015_8th_read.sort_values("AVG_SCORE", ascending = False)

f, ax = plt.subplots(figsize = (10, 15))  
   
sns.set_color_codes("pastel")
ax = sns.barplot(x = "TOTAL_REV_PER_STUDENT", y = "STATE", 
                 label = "REVENUE PER STUDENT", data = sort_8th_read, color = "r", ax = ax)

sns.set_color_codes("muted")
ax = sns.barplot(x = "INSTRUCTION_EXPENDITURE_PER_STUDENT", y = "STATE", 
                 label = "INSTRUCTION EXPENDITURE", data = sort_8th_read, color = "r", ax = ax)

handles, labels = ax.get_legend_handles_labels()

ax.legend(ncol = 1, loc = "upper right", frameon = True, edgecolor = "w")
ax.set(xlim = (0, 30500), ylabel = "",
       xlabel = "DOLLAR PER STUDENT")
plt.title("Revenue Per Student Sorted by Avg Reading Scores\nYear 2015", fontsize = 14, fontweight = "bold")

sns.despine(bottom = True)
plt.tight_layout(h_pad = 1)

plt.savefig("Output_images/Rev_Exp_Read_AvgScore")


In [ ]:
# Sorted by 8th grade Math avg score
sns.set_theme(color_codes = True)
ax_read = sns.catplot(x ="AVG_SCORE", y = "STATE", data = sort_8th_read, color = "brown", 
                      height = 15, aspect = 0.5)
plt.grid(which = "both", ls ="-")

ax_read.set_xlabels("Sorted Average Score - Reading", fontsize = 12)
plt.title("Sorted by Reading Average Scores\nYear 2015", fontsize = 14, fontweight = "bold")

plt.savefig("Output_images/AvgScore_Read_Sort")


### 2015 Funding vs Average Math Scores

In [ ]:
# Total revenue vs Instruction expenditure per student
# Sorted by 8th grade Math avg score

year_2015_8th_math = year_2015[(year_2015['TEST_YEAR'] == 8) & (year_2015['TEST_SUBJECT'] == "Mathematics")]
sort_8th_math = year_2015_8th_math.sort_values("AVG_SCORE", ascending = False)

f, ax = plt.subplots(figsize = (10, 15))  
   
sns.set_color_codes("pastel")
ax = sns.barplot(x = "TOTAL_REV_PER_STUDENT", y = "STATE", 
                 label = "REVENUE PER STUDENT", data = sort_8th_math, color = "b", ax = ax)

sns.set_color_codes("muted")
ax = sns.barplot(x = "INSTRUCTION_EXPENDITURE_PER_STUDENT", y = "STATE", 
                 label = "INSTRUCTION EXPENDITURE PER STUDENT", data = sort_8th_math, color = "b", ax = ax)

handles, labels = ax.get_legend_handles_labels()

ax.legend(ncol = 1, loc = "center right", frameon = True, edgecolor = "w", borderaxespad = 0)
ax.set(xlim = (0, 30500), ylabel = "",
       xlabel = "DOLLAR PER STUDENT")
plt.title("Revenue Per Student Sorted by Avg Math Scores\nYear 2015", 
          fontsize = 14, fontweight = "bold")

sns.despine(bottom = True)
plt.tight_layout(h_pad = 1)

plt.savefig("Output_images/Rev_Exp_Math_AvgScore")


In [ ]:
# Sorted by 8th grade Math avg score
sns.set_theme(color_codes = True)
ax_math = sns.catplot(x ="AVG_SCORE", y = "STATE", data = sort_8th_math, color = "royalblue",
                      height = 15, aspect = 0.5)
plt.grid(which = "both", ls = "-")

ax_math.set_xlabels("Sorted Average Score - Math", fontsize = 12)
plt.title("Sorted Math Average Scores\nYear 2015", 
          fontsize = 14, fontweight = "bold")

plt.savefig("Output_images/AvgScore_Math_Sort)


### 2015 Total Reveue per Student vs Average Scores

In [ ]:
# Total revenue vs 8th grade Reading scores
sns.set_theme(color_codes = True)

f, ax = plt.subplots(figsize = (8,8))
sns.despine(f, left = False, bottom = False)

slope, intercept, r_value, p_value, std_err = st.linregress(year_2015_8th_read["TOTAL_REV_PER_STUDENT"],
                                                            year_2015_8th_read["AVG_SCORE"])

sns.scatterplot(x = "TOTAL_REV_PER_STUDENT", y = "AVG_SCORE", sizes = (1, 8), 
                data = year_2015_8th_read, color = "brown", ax = ax, legend = "auto")

plt.xlabel("Revenue Per Student\n(Dollar)", fontsize = 12)
plt.ylabel("Average Score - Reading", fontsize = 12)
plt.title("Revenue Per Student vs Reading Average Scores\nYear 2015", 
          fontsize = 14, fontweight = "bold")
ax.set(xlim = (5000, 30500), ylim = (230, 305))

print(f"The r-value is {r_value}")
print(f"The p-value is {p_value}")

plt.tight_layout()
plt.savefig("Output_images/2015_Rev_vs_Reading")


In [ ]:
# Total revenue vs 8th grade Math scores
sns.set_theme(color_codes = True)

f, ax = plt.subplots(figsize = (8,8))
sns.despine(f, left = False, bottom = False)


slope, intercept, r_value, p_value, std_err = st.linregress(year_2015_8th_math["TOTAL_REV_PER_STUDENT"],
                                                            year_2015_8th_math["AVG_SCORE"])

sns.scatterplot(x = "TOTAL_REV_PER_STUDENT", y = "AVG_SCORE", sizes = (1, 8), 
                data = year_2015_8th_math, color = "royalblue", ax = ax, legend = "auto")

plt.xlabel("Revenue Per Student\n(Dollar)", fontsize = 12)
plt.ylabel("Average Score - Math", fontsize = 12)
plt.title("Revenue Per Student vs Math Average Scores\nYear 2015", 
          fontsize = 14, fontweight = "bold")
ax.set(xlim = (5000, 30500), ylim = (230, 305))

print(f"The r-value is {r_value}")
print(f"The p-value is {p_value}")

plt.tight_layout()
plt.savefig("Output_images/2015_Rev_vs_Math")


### 2015 Intruction Expenditure Per Student vs Average Scores
#### Linear regression analysis

In [ ]:
# Linear regression for 8th grade Reading vs Spending per Student
sns.set_theme(color_codes = True)
f, ax = plt.subplots(figsize = (8,8))
sns.despine(f, left = False, bottom = False)

slope, intercept, r_value, p_value, std_err = st.linregress(year_2015_8th_read["INSTRUCTION_EXPENDITURE_PER_STUDENT"],
                                                            year_2015_8th_read["AVG_SCORE"])


line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq, (12000, 263), fontsize = 15, color = "darkred", fontweight = "bold")
ax.set(xlim = (3900, 16100), ylim = (230, 305))

sns.regplot(data = year_2015_8th_read, x = "INSTRUCTION_EXPENDITURE_PER_STUDENT", y = "AVG_SCORE", 
            line_kws = {"color": "coral", "label":"y = {0:.1f}x+{1:.1f}".format(slope,intercept)},
            scatter_kws = {"color": "brown", "edgecolor": "w"})

plt.xlabel("Instruction Expenditure Per Student\n(Dollar)", fontsize = 12)
plt.ylabel("Average Score - Reading", fontsize = 12)
plt.title("Instruction Expenditure Per Student vs Reading Average Scores\nYear 2015", 
          fontsize = 14, fontweight = "bold")

print(f"The r-value is {r_value}")
print(f"The p-value is {p_value}")

plt.tight_layout()
plt.savefig("Output_images/Reg_2015_Exp_Read")


In [ ]:
# Linear regression for 8th grade Math vs Spending per Student
sns.set_theme(color_codes = True)
f, ax = plt.subplots(figsize = (8,8))
sns.despine(f, left = False, bottom = False)

slope, intercept, r_value, p_value, std_err = st.linregress(year_2015_8th_math["INSTRUCTION_EXPENDITURE_PER_STUDENT"],\
                                                               year_2015_8th_math["AVG_SCORE"])

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq, (12000, 283), fontsize = 15, color = "navy", fontweight = "bold")
ax.set(xlim = (3900, 16100), ylim = (230, 305))

sns.regplot(data = year_2015_8th_math, x = "INSTRUCTION_EXPENDITURE_PER_STUDENT", y = "AVG_SCORE", 
            line_kws = {"color": "teal", "label":"y = {0:.1f}x+{1:.1f}".format(slope,intercept)},
            scatter_kws = {"color": "darkblue", "edgecolor": "w"})

plt.xlabel("Instruction Expenditure Per Student\n(Dollar)", fontsize = 12)
plt.ylabel("Average Score - Math", fontsize = 12)
plt.title("Instruction Expenditure Per Student vs Math Average Scores\nYear 2015", 
          fontsize = 14, fontweight = "bold")

print(f"The r-value is {r_value}")
print(f"The p-value is {p_value}")

plt.tight_layout()
plt.savefig("Output_images/Reg_2015_Exp_Math")


### Instruction Expenditure Per Student vs Average Scores - All Years

In [ ]:
# All year 8th grade Math 
all_year_math = merged_data_df.loc[(merged_data_df['TEST_SUBJECT'] == 'Mathematics') & \
                                   (merged_data_df['TEST_YEAR'] == 8)]

all_year_math.head()

In [ ]:
# All year 8th grade Reading
all_year_reading = merged_data_df.loc[(merged_data_df['TEST_SUBJECT'] == 'Reading') & \
                                      (merged_data_df['TEST_YEAR'] == 8)]

all_year_reading.head()

In [ ]:
# Linear regression for 8th grade Reading vs Spending per Student across the years
sns.set_theme(color_codes = True)
f, ax = plt.subplots(figsize = (8,8))
sns.despine(f, left = False, bottom = False)

slope, intercept, r_value, p_value, std_err = st.linregress(all_year_reading["INSTRUCTION_EXPENDITURE_PER_STUDENT"],
                                                            all_year_reading["AVG_SCORE"])

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq, (12000, 267), fontsize = 15, color = "darkred", fontweight = "bold")
ax.set(xlim = (2500, 16100), ylim = (230, 305))

sns.regplot(data = all_year_reading, x = "INSTRUCTION_EXPENDITURE_PER_STUDENT", y = "AVG_SCORE", 
            line_kws = {"color": "coral", "label":"y = {0:.1f}x+{1:.1f}".format(slope,intercept)},
            scatter_kws = {"color": "brown", "edgecolor": "w"})

plt.xlabel("Instruction Expenditure Per Student\n(Dollar)", fontsize = 12)
plt.ylabel("Average Score - Reading", fontsize = 12)
plt.title("Instruction Expenditure Per Student vs Reading Average Scores\nAll Years", 
          fontsize = 14, fontweight = "bold")

print(f"The r-value is {r_value}")
print(f"The p-value is {p_value}")

plt.tight_layout()
plt.savefig("Output_images/Reg_All_Exp_Read")


In [ ]:
# Linear regression for 8th grade Math vs Spending per Student across the years
sns.set_theme(color_codes = True)
f, ax = plt.subplots(figsize = (8,8))
sns.despine(f, left = False, bottom = False)

slope, intercept, r_value, p_value, std_err = st.linregress(all_year_math["INSTRUCTION_EXPENDITURE_PER_STUDENT"],
                                                            all_year_math["AVG_SCORE"])

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq, (12000, 287), fontsize = 15, color = "navy", fontweight = "bold")
ax.set(xlim = (2300, 16100), ylim = (230, 305))

sns.regplot(data = all_year_math, x = "INSTRUCTION_EXPENDITURE_PER_STUDENT", y = "AVG_SCORE", 
            line_kws = {"color": "teal", "label":"y = {0:.1f}x+{1:.1f}".format(slope,intercept)},
            scatter_kws = {"color": "darkblue", "edgecolor": "w"})

plt.xlabel("Instruction Expenditure Per Student\n(Dollar)", fontsize = 12)
plt.ylabel("Average Score - Math", fontsize = 12)
plt.title("Instruction Expenditure Per Student vs Math Average Scores\nAll Years", 
          fontsize = 14, fontweight = "bold")

print(f"The r-value is {r_value}")
print(f"The p-value is {p_value}")

plt.tight_layout()
plt.savefig("Output_images/Reg_All_Exp_Math")


### Data Output

In [ ]:
# output file (csv)
merged_data_df.to_csv("Output_Data/merged_data_df.csv", index = False, header = True)
all_year_math.to_csv("Output_Data/all_year_math.csv", index = False, header = True)
all_year_reading.to_csv("Output_Data/all_year_reading.csv", index = False, header = True)
